In [1]:
import os
import sys
import random
from pathlib import Path
import osgeo  #Needed for use on Windows only
import matplotlib.pyplot as plt
from rasterio import plot
from rasterio.plot import show
import geopandas as gpd
from datetime import datetime
import sqlalchemy as sa
import pandas as pd
%matplotlib inline

In [2]:
sys.path.append(r"../../src/collectcube")
from samplegen import *
from db import *

In [3]:
out_dir = 'C:/GISprojects/ParaguayTraining'
local_db_path = os.path.join(out_dir, 'py_training.db')
## reference .tif with target crs and resolution
ref_ras = Path("../../data/CEL_py/samp_ras.tif")

res = 10

## Format existing sample points

In [4]:
old_pts = pd.read_csv(os.path.join(out_dir,'train_fordb.csv'))
old_pts_df = gpd.GeoDataFrame(old_pts,geometry = gpd.points_from_xy(old_pts.cent_X, old_pts.cent_Y, crs="EPSG:8858"))
old_pts_df.to_crs('4326')
old_pts_df['cent_long'] = old_pts_df['geometry'].x
old_pts_df['cent_lat'] = old_pts_df['geometry'].y
old_pts_df.to_crs("EPSG:8858")
old_pts_df.head()

,PID,Center,cent_Y,cent_X,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2,geometry
0,0000001_0,1,-2775277,2813634,-2775277.0,2813634.0,0,NaN,1,0,CAN - unverified in GE,0.756024,0.53,POINT (2813634.000 -2775277.000)
1,0000002_0,1,-2776460,2813534,-2776460.0,2813534.0,0,NaN,2,0,CAN - unverified in GE,0.600634,0.34,POINT (2813534.000 -2776460.000)
2,0000003_0,1,-2776486,2814015,-2776486.0,2814015.0,0,NaN,3,0,CAN - unverified in GE,0.172018,0.59,POINT (2814015.000 -2776486.000)
3,0000004_0,1,-2776486,2814457,-2776486.0,2814457.0,0,NaN,4,0,CAN - unverified in GE,0.438246,0.79,POINT (2814457.000 -2776486.000)
4,0000005_0,1,-2776489,2813057,-2776489.0,2813057.0,0,NaN,5,0,CAN - unverified in GE,0.684106,0.30,POINT (2813057.000 -2776489.000)


In [5]:
ptfile = os.path.join(out_dir,'training_pts_cent.shp')
centered_pts = move_points_to_pixel_centroids(old_pts_df, ref_ras, write_pts=True, ptsout=ptfile)

ref_ras has crs:EPSG:8858


In [6]:
all_pts = get_full_point_file(centered_pts, pt_file_out=None, res=res, lastpt=-1, write_pts=False)

<class 'geopandas.geodataframe.GeoDataFrame'>
there are 229304 neighbor pixels
there are 257967 total pixels


In [7]:
boxfile = os.path.join(out_dir,'training_boxes.shp')
make_pixel_boxes_from_pts(all_pts, boxfile, res, write_boxes=True)

,geometry
0,"POLYGON ((2813645.000 -2775295.000, 2813645.00..."
1,"POLYGON ((2813545.000 -2776475.000, 2813545.00..."
2,"POLYGON ((2814025.000 -2776505.000, 2814025.00..."
3,"POLYGON ((2814475.000 -2776505.000, 2814475.00..."
4,"POLYGON ((2813075.000 -2776505.000, 2813075.00..."
...,...
257962,"POLYGON ((3078445.000 -3285655.000, 3078445.00..."
257963,"POLYGON ((3078465.000 -3285655.000, 3078465.00..."
257964,"POLYGON ((3078445.000 -3285665.000, 3078445.00..."
257965,"POLYGON ((3078455.000 -3285665.000, 3078455.00..."


In [8]:
pts_info = make_pixel_table(all_pts,samp_group=None)
pts_info.drop(columns=['sampgroup'], inplace=True)
pts_info2 = pd.merge(pts_info,old_pts[['PID0','sampgroup','rand','rand2']],on='PID0', how='inner')
pts_info2.head()

,PID,Center,cent_X,cent_Y,cent_lat,cent_long,ransamp,checked,PID0,PID1,sampgroup,rand,rand2
0,0000001_0,1,2813540.0,-2776470.0,-21.878185,-59.587592,1,0,1,0,CAN - unverified in GE,0.756024,0.53
1,0000001_1,0,2813530.0,-2776460.0,-21.878104,-59.587708,1,0,1,1,CAN - unverified in GE,0.756024,0.53
2,0000001_2,0,2813540.0,-2776460.0,-21.878104,-59.587599,1,0,1,2,CAN - unverified in GE,0.756024,0.53
3,0000001_3,0,2813550.0,-2776460.0,-21.878104,-59.587491,1,0,1,3,CAN - unverified in GE,0.756024,0.53
4,0000001_4,0,2813530.0,-2776470.0,-21.878185,-59.587700,1,0,1,4,CAN - unverified in GE,0.756024,0.53


In [9]:
make_pixel_table_in_db(pts_info2, local_db_path, treat_existing='append')
table_check = check_table(local_db_path,'pixels')

              PID  Center     cent_X     cent_Y   cent_lat  cent_long  \
252130  0028662_4       0  3078440.0 -3285650.0 -26.027778 -56.230277   
252131  0028662_5       0  3078460.0 -3285650.0 -26.027778 -56.230057   
252132  0028662_6       0  3078440.0 -3285660.0 -26.027860 -56.230266   
252133  0028662_7       0  3078450.0 -3285660.0 -26.027860 -56.230156   
252134  0028662_8       0  3078460.0 -3285660.0 -26.027860 -56.230046   

        ransamp  checked   PID0  PID1 sampgroup      rand     rand2  
252130        1        0  28662     4     GE_KW  0.007343  0.227759  
252131        1        0  28662     5     GE_KW  0.007343  0.227759  
252132        1        0  28662     6     GE_KW  0.007343  0.227759  
252133        1        0  28662     7     GE_KW  0.007343  0.227759  
252134        1        0  28662     8     GE_KW  0.007343  0.227759  


## Make other db tables

In [10]:
lc_lut = Path("../../data/Class_LUT.csv")
make_LC_table_from_lut(lc_lut, local_db_path, treat_existing='fail')
make_simp_LC_table(lc_lut, local_db_path, [0,30,35,40,80,98], treat_existing='fail')
make_db(local_db_path)

added LC5 and PixVar tables to db


## checks:

In [11]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as conn:
    lc5_table = Table('LC5', MetaData(), autoload_with=engine)
    rp = conn.execute(lc5_table.select())
results = rp.fetchall() 
print(results)

[(0, ' ------'), (10, 'NoVeg'), (20, 'LowVeg'), (40, 'MedVeg'), (50, 'HighVeg'), (70, 'Trees'), (99, 'Unknown')]


## updating db (should not need if starting new db)

In [6]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE PixelVerification ADD COLUMN entry_lev INTEGER")
## filling column in existing table
#c.execute("UPDATE PixelVerification SET entry_lev = 4 WHERE entry_lev IS NULL")
con.commit()
c.close()

In [19]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as con:
    df = pd.read_sql_table('PixelVerification', con)
df.head()

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev,FieldWidth,source
0,1,0_0,0,0,2000-01-01,0,0,8,None,None,...,None,None,None,None,None,None,None,None,None,None


## add first record

In [13]:
## first record:
#testDate = datetime(2000,1,1)
#with engine.connect() as conn:
#    pixvar = Table('PixelVerification', MetaData(), autoload_with=engine)
#    query = sa.insert(pixvar).values(PID='0_0', PID0=0, PID1=0, imgDate=testDate,LC5=0,LC=0,HOMONBHD9=8) 
#    go = conn.execute(query)
#    conn.commit()
#print(go.inserted_primary_key)

(1,)


### Add existing records

In [20]:
#conn = sqlite3.connect(local_db_path, isolation_level=None,
#    detect_types=sqlite3.PARSE_COLNAMES)
#addfile = 'C:/GISprojects/ParaguayTraining/samp_fordb.csv'
#pd.read_csv(addfile).to_sql('PixelVerification', conn, if_exists='append', index=False)

28663

## cleaning

In [22]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2022-01-01' WHERE imgDate ='0'")
con.commit()
c.close()
'''
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("DELETE FROM PixelVerification WHERE PID = '507_0'")
con.commit()
c.close()
'''